In [ ]:
%matplotlib inline
import openpathsampling as paths
import numpy as np
from toy_plot_helpers import ToyPlot

In [ ]:
import openpathsampling.toy_dynamics as toys
plot = ToyPlot()
pes = (
    toys.toy_pes.OuterWalls([1.0, 1.0], [0.0, 0.0]) +
    toys.toy_pes.Gaussian(-0.7, [12.0, 12.0], [-0.5, 0.5]) +
    toys.toy_pes.Gaussian(-0.7, [12.0, 12.0], [-0.5, -0.5]) +
    toys.toy_pes.Gaussian(-0.7, [12.0, 12.0], [0.5, -0.5])
)

plot.contour_range = np.arange(-1.5, 1.0, 0.1)

topology=paths.ToyTopology(
    n_spatial = 2,
    masses =[1.0, 1.0],
    pes = pes
)

template = paths.Snapshot(
    coordinates=np.array([[-0.5, -0.5]]), 
    velocities=np.array([[0.0,0.0]]),
    potential_energy = 0.0, # Important to set energies dimensionless otherwise kcal/mol is default
    kinetic_energy = 0.0,
    topology=topology
)

integ = toys.toy_integrators.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5)

options={
    'integ' : integ,
    'n_frames_max' : 5000,
    'nsteps_per_frame' : 10
}

toy_eng = toys.toy_engine.ToyEngine(
    options=options,
    template=template
)
toy_eng.initialized = True

toy_eng.current_snapshot = template
paths.PathMover.engine = toy_eng

In [ ]:
plot.add_pes(pes)
plot.plot()

In [ ]:
def xval(snapshot):
    return snapshot.xyz[0][0]

def xprime(snapshot):
    # this only exists until we set up the ability for the order parameter to decrease
    return -snapshot.xyz[0][0]

def yval(snapshot):
    return snapshot.xyz[0][1]
    
opX = paths.CV_Function(name="opX", fcn=xval)
opY = paths.CV_Function(name="opY", fcn=yval)
opXprime = paths.CV_Function(name="opXprime", fcn=xprime)

In [ ]:
x_under_min = paths.CVRangeVolume(opX, float("-inf"), -0.3)
x_over_max = paths.CVRangeVolume(opX, 0.3, float("inf")) 
y_under_min = paths.CVRangeVolume(opY, float("-inf"), -0.3)
y_over_max = paths.CVRangeVolume(opY, 0.3, float("inf")) 

stateA = x_under_min & y_under_min
stateB = x_over_max & y_under_min
stateC = x_under_min & y_over_max

In [ ]:
#plot.add_states([stateA, stateB, stateC])
#plot.plot()

In [ ]:
interfacesAB = paths.VolumeFactory.CVRangeVolumeSet(opX, float("-inf"), [-0.3, -0.2, -0.1, 0.0])
interfacesAC = paths.VolumeFactory.CVRangeVolumeSet(opY, float("-inf"), [-0.3, -0.2, -0.1, 0.0])
interfacesBA = paths.VolumeFactory.CVRangeVolumeSet(opXprime, float("-inf"), [-0.3, -0.2, -0.1, 0.0])

In [ ]:
tisAB = paths.TISTransition(stateA, stateB, interfacesAB, orderparameter=opX, name="A->B")
tisAC = paths.TISTransition(stateA, stateC, interfacesAC, orderparameter=opY, name="A->C")
tisBA = paths.TISTransition(stateB, stateA, interfacesBA, orderparameter=opXprime, name="B->A")

In [ ]:
mistis = paths.MISTISNetwork([tisAB, tisAC, tisBA])

In [ ]:
import logging.config
logging.config.fileConfig("debug_logging.conf", disable_existing_loggers=False)

In [ ]:
snapA = paths.Snapshot(
    coordinates=[[-0.5, -0.5]],
    velocities=[[0.5, 0.0]]
)
init_AB = paths.FullBootstrapping(
    transition=tisAB, 
    snapshot=snapA, 
    engine=toy_eng, 
    forbidden_states=[stateC]
).run()

In [ ]:
snapA = paths.Snapshot(
    coordinates=[[-0.5, -0.5]],
    velocities=[[0.0, 0.5]]
)
init_AC = paths.FullBootstrapping(
    transition=tisAC, 
    snapshot=snapA, 
    engine=toy_eng, 
    forbidden_states=[stateB]
).run()

In [ ]:
snapB = paths.Snapshot(
    coordinates=[[0.5, -0.5]],
    velocities=[[-0.5, 0.0]]
)
init_BA = paths.FullBootstrapping(
    transition=tisBA, 
    snapshot=snapB, 
    engine=toy_eng, 
    forbidden_states=[stateC]
).run()

In [ ]:
plot.plot([s.trajectory for s in list(init_AB)+list(init_AC)+list(init_BA)])

In [ ]:
# TODO: it would be really nice if all of this could be handled somewhat automatically
samples = []
trans2trajs = {tisAB : init_AB, tisAC : init_AC, tisBA : init_BA}
for trans in trans2trajs.keys():
    new_trans = mistis.transition_to_sampling[trans]
    for i in range(len(new_trans.ensembles)):
        samples.append(paths.Sample(
                trajectory=trans2trajs[trans][i].trajectory,
                ensemble=new_trans.ensembles[i],
                replica=len(samples)
        ))
sset = paths.SampleSet(samples)
sset.sanity_check()

outer_sset = paths.SampleSet([init.samples[-1] for init in [init_AB, init_BA]])
outer_sample = paths.Sample(
    replica=len(sset),
    trajectory=init_AB.samples[-1].trajectory,
    ensemble=mistis.ms_outers[0]
)
sset = sset.apply_samples([outer_sample])
sset.sanity_check()

minus_samples = []
for minus in mistis.minus_ensembles:
    for partial_traj in [s.trajectory for s in sset]:
        if minus._segment_ensemble(partial_traj):
            break
    #print minus._segment_ensemble(partial_traj)
    good_sample = False
    while not good_sample:
        samp = minus.populate_minus_ensemble(
            partial_traj=partial_traj,
            minus_replica_id=-mistis.minus_ensembles.index(minus)-1,
            engine=toy_eng
        )
        good_sample = samp.ensemble(samp.trajectory)
    minus_samples.append(samp)

sset = sset.apply_samples(minus_samples)
sset.sanity_check()

In [ ]:
#logging.config.fileConfig("debug_logging.conf", disable_existing_loggers=False)
storage = paths.storage.Storage("mistis.nc", "w", template=template)

In [ ]:
storage.save(mistis)
storage.sync()

In [ ]:
scheme = paths.DefaultScheme(mistis)

In [ ]:
mistis_calc = paths.PathSampling(
    storage=storage,
    engine=toy_eng,
    move_scheme=scheme.move_decision_tree(),
    globalstate=sset
)
mistis_calc.save_frequency = 100

In [ ]:
storage.save(scheme)
storage.sync()

In [ ]:
import logging.config
logging.config.fileConfig("logging.conf", disable_existing_loggers=False)

mistis_calc.run_until(250)